In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [2]:
with open('News.txt', encoding="utf8") as f:
    lines = f.read()

In [3]:
doc1 = nlp(lines)

for ent in doc1.ents:
    if ent.label_ == "GPE" :
        print(ent.text)

Ukraine
Ukraine
Kherson
Kherson
Russia
Ukraine
Russia
Ukraine
London
Russia
the west bank
Ukrainian
Kherson
Kakhova
Ukraine
Kherson
Zaporizhzhia
Ukraine
Moscow
US
Ukraine
agoGermany
Israel
Ukraine
Dagestan
UK


In [4]:
doc1.ents

(Ukraine,
 Russian,
 Dnipro River,
 Ukraine,
 The Guardian
 Skip,
 Guardian - Back,
 NewsWorld,
 crisisUkraineEnvironmentScienceGlobal,
 OpinionThe Guardian,
 CultureBooksMusicTV & radioArt & designFilmGamesClassicalStageLifestyleView,
 & sexHealth & fitnessHome & gardenWomenMenFamilyTravelMoneySearch,
 SearchSupport,
 ArchiveGuardian Puzzles,
 LicensingThe Guardian,
 appVideoPodcastsPicturesNewslettersToday,
 the GuardianThe ObserverGuardian,
 ArchiveGuardian Puzzles,
 Ukrainian,
 the Dnipro River,
 Kherson,
 last month,
 Alex Babenko/,
 the Dnipro River,
 Kherson,
 last month,
 Alex Babenko,
 Russian,
 Dnipro RiverBeachhead,
 Kherson,
 Russia,
 Tom Burgis,
 KhersonFri,
 GMTLast,
 Fri 3 Nov 2023 20.40,
 Ukrainian,
 the Dnipro River,
 Russian,
 Ukraine,
 Kherson,
 Ukrainian,
 Crimea,
 Russia,
 2014,
 two,
 Ukrainian,
 the east bank,
 mid-October,
 Russian,
 Ukrainian,
 Volodymyr Saldo,
 Putin,
 Kherson,
 Russian,
 Ukrainians,
 Kherson,
 Ukraine,
 Russian,
 Russian,
 fortificationsThis 

In [63]:
doc2 = nlp("the Israel-Hamas war")
doc3 = nlp("Anthony Albanese’s three-day tour of Shanghai and Beijing")

print(doc2.similarity(doc3))

0.5912129800317639


C:\Users\sures\AppData\Local\Temp\ipykernel_29416\1013727239.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(doc2.similarity(doc3))


In [15]:
doc4 = """Computer: Based on the keywords in the title and body of the text, I would categorize the article as belonging to the "Politics" class.

The Guardian is a UK newspaper and they are using their own category scheme which is different from the one used by the user. The user has categorized the article under "Politics" but the Guardian has categorized it under "Israel". So there is a difference in how the article is being categorized by the two entities.. """

for tok in nlp(doc4):
    print(str(tok).lower())

computer
:
based
on
the
keywords
in
the
title
and
body
of
the
text
,
i
would
categorize
the
article
as
belonging
to
the
"
politics
"
class
.



the
guardian
is
a
uk
newspaper
and
they
are
using
their
own
category
scheme
which
is
different
from
the
one
used
by
the
user
.
the
user
has
categorized
the
article
under
"
politics
"
but
the
guardian
has
categorized
it
under
"
israel
"
.
so
there
is
a
difference
in
how
the
article
is
being
categorized
by
the
two
entities
..


# 2. Classify to broad dimentions

In [25]:
# Built a multilabel classification system using SpaCy (build and train the modek)

import pandas as pd

import sys
import os
import spacy
from spacy import displacy

from tqdm.auto import tqdm
from spacy.tokens import DocBin

In [3]:
def make_docs(train_data):

    train_data = train_data.drop(columns=["Id"])

    data = tuple(zip(train_data['Title'].tolist(), train_data['Label'].tolist())) 

    nlp = spacy.load("en_core_web_sm")

    docs = []

    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if (label=='Nature'):
            doc.cats['Nature'] = 1
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Politics'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 1
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Entertainment'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 1
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Economics'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 1
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 0
        elif (label=='Culture'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 1
            doc.cats['Science']  = 0
        elif (label=='Science'):
            doc.cats['Nature'] = 0
            doc.cats['Politics'] = 0
            doc.cats['Entertainment']  = 0
            doc.cats['Economics']  = 0
            doc.cats['Culture']  = 0
            doc.cats['Science']  = 1        

        docs.append(doc)

    return docs,train_data

In [68]:
df = pd.read_csv("C:\Work\Programming\Hackathons\Diplomacy software\Dimensions.csv")
train = df.sample(frac=0.7, replace=False, random_state=1)

train_docs, train_data = make_docs(train)

doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./textcat_data/textcat_train.spacy")

test = df.sample(frac=0.3, replace=False, random_state=1)

test_docs, train_data = make_docs(test)

doc_bin = DocBin(docs=test_docs)
doc_bin.to_disk("./textcat_data/textcat_valid.spacy")

100%|██████████| 12/12 [00:00<00:00, 512.69it/s]


In [69]:
train_data

,Title,Label
2,NBA Playoffs: Lakers and Clippers Face Off in ...,Entertainment
30,Exotic Destinations for Your Next Adventure Va...,Entertainment
3,President Signs Historic Climate Change Legisl...,Politics
21,Scientific Expedition Discovers New Species in...,Science
26,High School Robotics Team Wins International C...,Politics
28,Mental Health Awareness Month: Initiatives to ...,Science
22,Innovative Schools Implement Technology-Driven...,Science
36,Wildfires Threaten Communities in the Western ...,Nature
19,Researchers Develop Promising Treatment for Al...,Science
25,Education Policy Reforms Aim to Improve Studen...,Politics


In [70]:
!python -m spacy init fill-config ./textcat_base_config.cfg ./textcat_config.cfg

✔ Auto-filled config with all values
✔ Saved config
textcat_config.cfg
You can now add your data and train your pipeline:
python -m spacy train textcat_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [71]:
!python -m spacy train textcat_config.cfg --verbose --output ./textcat_output --paths.train textcat_data/textcat_train.spacy --paths.dev textcat_data/textcat_valid.spacy

[2023-10-27 22:02:30,992] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
[2023-10-27 22:02:31,180] [INFO] Set up nlp object from config
[2023-10-27 22:02:31,187] [DEBUG] Loading corpus from path: textcat_data\textcat_valid.spacy
[2023-10-27 22:02:31,192] [DEBUG] Loading corpus from path: textcat_data\textcat_train.spacy
[2023-10-27 22:02:31,192] [INFO] Pipeline: ['tok2vec', 'textcat']
[2023-10-27 22:02:31,192] [INFO] Created vocabulary
[2023-10-27 22:02:31,587] [WARNING] [W112] The model specified to use for initial vectors (en_core_web_sm) has no vectors. This is almost certainly a mistake.
[2023-10-27 22:02:31,587] [INFO] Added vectors: en_core_web_sm
[2023-10-27 22:02:31,587] [INFO] Finished initializing nlp object
[2023-10-27 22:02:31,852] [INFO] Initialized pipeline components: ['tok2vec', 'textcat']
[2023-10-27 22:02:31,864] [DEBUG] Loading corpus from path: textcat_data\textcat_valid.spacy
[2023-10-27 22:02:31,864] [DEBUG] Loading corpus from path: textcat_data\

ℹ Saving to output directory: textcat_output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ------------  ----------  ------
  0       0          0.00          0.14       37.26    0.37
 77     200         84.36          1.09       66.67    0.67
177     400        482.64          0.23       66.67    0.67
277     600         19.61          0.01       66.67    0.67
418     800          0.00          0.00       66.67    0.67
618    1000          0.00          0.00       66.67    0.67
818    1200          0.00          0.00       66.67    0.67
1018    1400          0.00          0.00       66.67    0.67
1218    1600          0.00          0.00       66.67    0.67
1418    1800          0.00          0

In [28]:
prompt = input("Enter a headline to analyse: ")
nlp_textcat = spacy.load(os.getcwd() + "\\textcat_output\\model-last")
docPred = nlp_textcat(prompt)
res = docPred.cats
print(res)
print(prompt)
print("The class of this headline is", max(res, key=res.get))

{'Nature': 0.6959365010261536, 'Politics': 2.370913716731593e-05, 'Entertainment': 2.5340261000650344e-09, 'Economics': 0.1244477927684784, 'Culture': 0.00027904147282242775, 'Science': 0.17931297421455383}
Singapore votes in favour of UN resolution to protect civilians, uphold humanitarian obligations in Gaza Strip
The class of this headline is Nature


# 3. Map onto seriousness of risk

# 4. Differentiate domesting and international events

In [7]:
lines = """
Taipei, Taiwan – After starting 2023 with a bang, China’s economy had a bumpy recovery over the past year.

The Chinese economy’s precarious footing looks set to continue into 2024, as deep-seated structural issues and Chinese President Xi Jinping’s consolidation of political control threaten to dampen growth.

China’s reopening after the lifting of its harsh “zero-COVID” restrictions in January coincided with challenging economic conditions overseas, as soaring inflation made consumers less inclined to buy Chinese goods.

At home, Chinese consumers were wary to start spending again after nearly two years of lockdowns and border closures.

In July, China bucked the global trend and entered a period of deflation, which it struggled to exit in the second half of the year.

Prices in November fell 0.5 percent year on year – the sharpest drop in three years.

China’s real estate crisis continued to roll on as more developers teetered on the brink of default and home sales remained at half their December 2020 levels – spelling trouble for an economy in which property accounts for around 30 percent of gross domestic product (GDP) and nearly 70 percent of household wealth.

While the International Monetary Fund (IMF) expects the Chinese economy to finish the year at 5.4 percent growth, economists predict a slowdown in 2024 and beyond amid structural problems such as record levels of debt and a low birth rate.

Foreign investors have voted with their pocketbooks.

China posted a foreign investment deficit of $11.8bn in the three months to September – the first time foreign businesses pulled more money out of the country than they put in since records began.
"""

# with open('News.txt', encoding="utf8") as f:
#     lines = f.read()

In [8]:
doc1 = nlp(lines)

places = []

for ent in doc1.ents:
    if ent.label_ == "GPE" :
        places.append(ent.text)

print(places)

['Taipei', 'Taiwan', 'China', 'China', 'China', 'China', 'China']


In [9]:
# This blocks initialised the geography module
from functools import partial
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="diplomacy-software")

geocode = partial(geolocator.geocode, language="en")

In [10]:
countries = {}

for place in places:
    try:
        country = geocode(place)[0].split(',')[-1].strip()

        if country not in countries:
            countries[country] = 1
        else:
            countries[country] += 1

    except TypeError:
        pass

countries

{'Taiwan': 2, 'China': 5}

In [11]:
total = sum(list(countries.values()))

threshold = total / len(countries)

relevant = []

for country in countries:
    
    if countries[country] > threshold:
        relevant.append(country)

print(relevant)

['China']


In [12]:
countries

{'Taiwan': 2, 'China': 5}